# Movies analysis using Python Pandas and Object-oriented Approach (OOP)

The code in this notebook analyses the movies dataset downloaded from https://www.kaggle.com/rounakbanik/the-movies-dataset by using the data from the `movies_metadata.csv` and `ratings.csv` files according to the requirements specified in the file [Data_Engineering_Task.txt](https://github.com/x4x3r/scaling-octo-eureka/blob/main/Data-Engineer_task.txt).

Refer to point **8. Movies analysis program using Pandas and object-oriented approach (OOP)** for the object-oriented (OOP) solution.

## Import the required libraries

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, date

In [ ]:
# Create directory named "data" and unzip the downloaded movies data
!mkdir '/home/user/Desktop/data_engineering_task/data'
!unzip 'archive.zip' -d '/home/user/Desktop/data_engineering_task/data'

In [7]:
# Check the unzipped data
%ls -l data/

total 921660
-rw-rw-r-- 1 user user 189917659 сеп 21  2019 credits.csv
-rw-rw-r-- 1 user user   6231943 сеп 21  2019 keywords.csv
-rw-rw-r-- 1 user user    989107 сеп 21  2019 links.csv
-rw-rw-r-- 1 user user    183372 сеп 21  2019 links_small.csv
-rw-rw-r-- 1 user user  34445126 сеп 21  2019 movies_metadata.csv
-rw-rw-r-- 1 user user 709550327 сеп 21  2019 ratings.csv
-rw-rw-r-- 1 user user   2438266 сеп 21  2019 ratings_small.csv


## Data exploration and  processing

The number of movies can be calculated from the `movies_metadata.csv` dataset

In [3]:
#  Read the movies_metadata.csv dataset into a pandas DataFrame
movies_df = pd.read_csv('data/movies_metadata.csv', header=0)

# Display the first 5 rows of the DataFrame to get a glimpse of the data
movies_df.head(5)

/tmp/ipykernel_6057/3685314342.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_df = pd.read_csv('data/movies_metadata.csv', header=0)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [6]:
movies_df['genres'].iloc[0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [ ]:
# Check which columns can be used for counting the movies
movies_df.columns

In [ ]:
# Determine the total number of rows in the movies metadata dataset
len(movies_df)

**Explanation**: There are 45466 rows in the movies_metadata.csv. `movies_df.columns` reveals that there are 2 possible id columns: 'id' and 'imdb_id'. Further inspection reveals that 'id' has more values and no NA values, while the index numbers in 'imdb_id' have 17 movies less. So, 'id' column might be appropriate for movie count and as an id column of the dataset.

In [ ]:
# Check the number of rows in the 'id' column that have NA values
movies_df['id'].isna().sum()

**Explanation**: There are no rows in the 'id' column with NA values, indicating that it is a complete and suitable column for further analysis.

In [ ]:
# Check the number of rows in the 'imdb_id' column that have NA values
movies_df['imdb_id'].isna().sum()

**Explanation**: There are 17 movies in the dataset that are not indexed by IMDB but still exist in the movies database on Kaggle. These movies have missing values in the 'imdb_id' column.

In [ ]:
# Display the 17 movies that have missing 'imdb_id' values
movies_df[movies_df['imdb_id'].isna()]

**Explanation**: The above DataFrame shows the 17 movies that do not have an 'imdb_id'. These movies might have limited information available, making it difficult to use them in further analysis.

Also, `pd.read_csv` returns **DtypeWarning: Columns (10) have mixed types**. Columns (10) is the 'id' column. We will try to filter these non-alphanumeric symbols to see what's in there.

In [ ]:
# Filter out the rows in the 'id' column that contain non-alphanumeric symbols
movies_df[~movies_df['id'].str.isalnum()]

**Explanation**: The 'id' column contains three rows with release dates instead of alphanumeric values. Since these rows do not provide much useful information and lack proper 'id' numbers, it is best to drop them from the DataFrame.

In [ ]:
# Dropping the non-alphanumeric rows
movies_df = movies_df[movies_df['id'].str.isalnum()]

Also, it's worthwile checking for duplicate rows. Here are they:

In [ ]:
# Check for duplicate rows in the 'id' column
movies_df[movies_df['id'].duplicated(keep=False)].sort_values('id')

**Explanation**: The above DataFrame displays the rows in the 'id' column that are duplicated. It helps identify any inconsistencies or redundant data.

In [ ]:
# Keep only the rows with unique 'id' numbers and update the dataframe
movies_df = movies_df.drop_duplicates(subset='id')

## 1. Load the dataset from a CSV file.

We can write all data processing operations as a single function

In [3]:
def data_processing(ratings_filepath, movies_filepath):
    """1. Load the dataset from a CSV file."""
    
    dtypes = {
        'genres': object,
        'release_date': str,
        'title': str,
    }
    
    # Read the movies_metadata.csv
    movie_columns = ['genres', 'id', 'release_date', 'title']
    movies_df = pd.read_csv(movies_filepath, header=0 , sep=',', \
                            parse_dates=True, \
                            usecols=movie_columns, \
                            dtype=dtypes)
    
    # Drop the rows in movies_df where 'id' column contains non-alphanumeric symbols
    movies_df = movies_df[movies_df['id'].str.isalnum()]
    
    # Keep only the rows with unique 'id' numbers
    movies_df = movies_df.drop_duplicates(subset='id')
    
    # Convert 'id' column to int data type
    movies_df['id'] = movies_df['id'].astype(int)
    
    # Set the 'id' column as the index of movies_df
    movies_df.set_index('id', inplace=True)  
    
    # Read the ratings.csv file into ratings_df DataFrame
    columns = ['movieId', 'rating']
    ratings_df = pd.read_csv(ratings_filepath, \
                        usecols=columns, \
                        index_col='movieId', \
                         dtype={'movieId':int, \
                        'rating':float})
    
    return ratings_df, movies_df

ratings_filepath = 'data/ratings.csv'
movies_filepath = 'data/movies_metadata.csv'

ratings_df, movies_df = data_processing(ratings_filepath, movies_filepath)

## 2. Print the number of movies in the dataset.

In [ ]:
# Print the number of unique movies
len(movies_df)

## 3. Print the average rating of all the movies.

In [ ]:
def average_rating_loops(file_path):
    """Calculate the average rating of all movies"""
        
    # Define the variables
    chunk_size= 5000
    
    # Initialize variables for average rating calculation
    total_sum = 0
    total_count = 0
    
    # Read the file in chunks to prevent memory overloading
    for chunk in pd.read_csv(file_path, chunksize=chunk_size):
        chunk_sum = chunk['rating'].sum()
        
        # Update the average values
        total_sum += chunk_sum
        total_count += len(chunk)
    
    # Calculate the average rating for all movies
    average_rating = total_sum/total_count
    
    # Print the average rating of all the movies
    print('Average movie rating: ' + str(np.round(average_rating, 2)))

file_path='data/ratings.csv'
average_rating_loops(file_path)

Another way is to use Pandas load only the columns necessary for calculating the average rating of all movies

In [ ]:
def average_rating(ratings):
    """Calculate the average rating of all movies"""
    
    average_rating = ratings_df['rating'].mean()
    print('Average movie rating: ' + str(np.round(average_rating, 2)))

average_rating(ratings_df)

## 4. Print the top 5 highest rated movies.

In [4]:
def top_5_movies(ratings, movies):
    """Print the top 5 highest rated movies"""
    
    # Group by 'movieId' and calculate the mean
    avg_ratings_df = ratings.groupby('movieId').mean('rating')
    
    # Merge the dataframes based on the index
    merged_df = pd.merge(avg_ratings_df, movies, left_index=True, right_index=True)
    
    # Get the top 5 movies
    top_5_movies = merged_df[['rating', 'title']].sort_values(by=['rating'], ascending=False).head(5)
    
    return top_5_movies

top5 = top_5_movies(ratings=ratings_df, movies=movies_df['title'])
top5

,rating,title
95977,5.0,The Man Behind The Gun
167666,5.0,Monster High: Escape from Skull Shores
130544,5.0,Palermo or Wolfsburg
129530,5.0,Brutal
164278,5.0,Harvey


This code can be improved by using weightened average

## 5. Print the number of movies released each year.

In [ ]:
def movies_per_year(movies):
    """Print the number of movies per year"""
    
    # Extract the year component from 'release_date' and store into a new column 'release_year'
    movies['release_year'] = pd.to_datetime(movies['release_date'], errors='coerce').dt.year
    
    # Dropping the na in 'release_year'
    movies['release_year'] = movies['release_year'].dropna()
    
    # Count the number of movies released each year
    movies_per_year = movies['release_year'].value_counts().sort_index()
    
    # Create a movies_per_year_df DataFrame from the Pandas Series
    movies_per_year_df = pd.DataFrame({'Year': movies_per_year.index, 'Count': movies_per_year.values})
    
    # Cast 'Year' column to integer data type
    movies_per_year_df['Year'] = movies_per_year_df['Year'].astype(int)
    
    return movies_per_year_df
    
n_movies = movies_per_year(movies_df)

n_movies

## 6. Print the number of movies in each genre.

In [ ]:
import ast

def movies_per_genre(movies):
    """Print the number of movies in each genre"""
    
    # Convert the string stored in column 'genres' to lists of dictionaries
    # movies_df['genres'] = movies_df['genres'].apply(ast.literal_eval)
    if isinstance(movies_df['genres'].iloc[0], str):
        movies_df['genres'] = movies_df['genres'].apply(ast.literal_eval)

    # Extract the relevant information from the nested dictionaries
    movies_df['genre_names'] = movies_df['genres'].apply(lambda x: [genre['name'] for genre in x])
    
    # Count the number of movies per genre and return a pandas.DataFrame
    movies_per_genre = movies_df['genre_names'].explode().value_counts()
    movies_per_genre.columns = ['Genre', 'Count']
    
    return movies_per_genre.to_frame().reset_index()

movies_per_genre(movies=movies_df)

## 7. Save the dataset to a JSON file.

In [ ]:
top_5_movies.to_json('path/to/top_5_movies.csv')
movies_per_year.to_json('path/to/movies_per_year.csv')
movies_per_genre.to_json('path/to/movies_per_genre.csv')

## 8. Program to analyse the movies dataset using Pandas and object-oriented approach (OOP)

This program can be used in this notebook and as a standalone application to print the results in the console window

In [1]:
import pandas as pd
import numpy as np
import ast
from datetime import datetime, date


class MoviesAnalisys():
    """Data analysis of the movies dataset"""


    def __init__(self, movies_filepath, ratings_filepath):
        """Initialize attributes"""
        
        # 1. Load the dataset from a CSV file.
        # Define the data types for each column
        dtypes_movies = {
            'genres': object,
            'release_date': str,
            'title': str,
        }
        
        # Read the movies_metadata.csv file
        movies_columns = ['genres', 'id', 'release_date', 'title']
        self.movies = pd.read_csv(movies_filepath, header=0, sep=',', \
                        parse_dates=True, \
                        usecols=movies_columns, \
                        dtype=dtypes_movies)
        
        # Drop the non-alphanumeric rows
        self.movies = self.movies[self.movies['id'].str.isalnum()]
        
        # Keep only the rows with unique 'id' numbers
        self.movies = self.movies.drop_duplicates(subset='id')
        
        # Convert 'id' column to int data type
        self.movies['id'] = self.movies['id'].astype(int)
        
        # Set the 'id' column as self.movies index
        self.movies.set_index('id', inplace=True)  
        
        ratings_columns = ['movieId', 'rating']
        self.ratings = pd.read_csv(ratings_filepath, \
                                usecols=ratings_columns, \
                                index_col='movieId', \
                                dtype={'movieId': int, 'rating':float})


    def count_movies(self):
        """2. Print the number of movies in the dataset."""

        print('Number of movies in the dataset: ' + str(len(self.movies)))


    def avg_rating(self):
        """3. Print the average rating of all the movies."""
        
        average_rating = self.ratings['rating'].mean()
        print('Average movie rating: ' + str(np.round(average_rating, 2)))


    def top_5_movies(self):
        """4. Print the top 5 highest rated movies."""
        
        # Calculate the average ratign by movieId
        avg_ratings_df = self.ratings.groupby('movieId').apply('mean', 'rating')
        
        # Convert 'id' column to int data type
        # self.movies['id'] = self.movies['id'].astype(int)

        # Merge the dataframes based on ratingsmovieId and id
        merged_df = pd.merge(avg_ratings_df, self.movies, left_index=True, right_index=True)
        
        # Calculate the top 5 movies
        top_5_movies = merged_df[['rating', 'title']].sort_values('rating', ascending=False).head(5)
        
        # print('Top 5 movies:' + str(top_5_movies))
        
        return top_5_movies    

    
    def movies_per_genre(self):
        """6. Print the number of movies in each genre."""
        
        # Convert the string stored in column 'genres' to lists of dictionaries
        if isinstance(self.movies['genres'].iloc[0], str):
            self.movies['genres'] = self.movies['genres'].apply(ast.literal_eval)
    
        # Extract the relevant information from the nested dictionaries
        self.movies['genre_names'] = self.movies['genres'].apply(lambda x: [genre['name'] for genre in x])
        
        # Count the number of movies per genre and convert to pandas.DataFrame
        movies_per_genre = self.movies['genre_names'].explode().value_counts()
        movies_per_genre.columns = ['Genre', 'Count']
    
        return movies_per_genre.to_frame().reset_index()    

    
    def movies_per_year(self):
        """5. Print the number of movies released each year."""
        
        # Extract the year component from 'release_date' and store into a new column 'release_year'
        self.movies['release_year'] = pd.to_datetime(self.movies['release_date'], errors='coerce').dt.year
        
        # Drop the `na` in 'release_year'
        self.movies['release_year'] = self.movies['release_year'].dropna()
        
        # Count the number of movies released each year
        movies_per_year = self.movies['release_year'].value_counts().sort_index()
        
        # Create a movies_per_year_df pandas.DataFrame from the pandas.Series
        movies_per_year_df = pd.DataFrame({'Year': movies_per_year.index, 'Count': movies_per_year.values})
        
        # Cast 'Year' column to integer data type
        movies_per_year_df['Year'] = movies_per_year_df['Year'].astype(int)
    
        return movies_per_year_df
    
    
    def save_to_json(self, result, filepath):
        """7. Save the dataset to a JSON file."""
        
        result.to_json(filepath)

In [2]:
movies_filepath = 'data/movies_metadata.csv'
ratings_filepath = 'data/ratings.csv'

# Create an instance of the MoviesAnalysis class
movies_analysis = MoviesAnalisys(movies_filepath, ratings_filepath)

In [3]:
# Print the number of movies in the dataset
movies_count = movies_analysis.count_movies()

Number of movies in the dataset: 45433


In [4]:
# Calculate the average rating of all movies
average_rating = movies_analysis.avg_rating()

Average movie rating: 3.53


In [5]:
# Extract the top 5 movies
top_5_movies_result = movies_analysis.top_5_movies()
top_5_movies_result

,rating,title
95977,5.0,The Man Behind The Gun
167666,5.0,Monster High: Escape from Skull Shores
130544,5.0,Palermo or Wolfsburg
129530,5.0,Brutal
164278,5.0,Harvey


In [6]:
# Calculate the number of movies in each genre
movies_per_genre_result = movies_analysis.movies_per_genre()
movies_per_genre_result

,genre_names,count
0,Drama,20244
1,Comedy,13176
2,Thriller,7619
3,Romance,6730
4,Action,6592
5,Horror,4671
6,Crime,4304
7,Documentary,3930
8,Adventure,3490
9,Science Fiction,3044


In [8]:
# Calculate the number of movies released each year
movies_per_year_result = movies_analysis.movies_per_year()
movies_per_year_result

,Year,Count
0,1874,1
1,1878,1
2,1883,1
3,1887,1
4,1888,2
...,...,...
130,2015,1904
131,2016,1604
132,2017,532
133,2018,5


In [9]:
# Store some of the reulting dataframes as JSON files
top_5_movies_result_filepath = 'results_json/top_5_movies.json'
movies_per_genre_result_filepath = 'results_json/movies_per_genre.json'
movies_per_year_result_filepath = 'results_json/movies_per_year.json'

movies_analysis.save_to_json(top_5_movies_result, top_5_movies_result_filepath)
movies_analysis.save_to_json(movies_per_genre_result, movies_per_genre_result_filepath)
movies_analysis.save_to_json(movies_per_year_result, movies_per_year_result_filepath)